# 1 вариант. 1 вопрос.

Вопрос: 1. Compare BERT and GPT-2: high-level description

Bert: маскированная языковая модель, которая состоит только из слоев энкодера. Она решает задачи предсказывания пропущенных(маскированных) слов (засчет двунаправленности модели) и определения логичности последовательности предложений (засчет большей связанности элементов в архитектуре). На вход получает следующие эмбеддинги: WordPiece tokenization embeddings (эмбеддинг токена (слово/часть)), sentence embeddings (отвечает за принадлежность к одному предложению), position embedding(позиция токена).

GPT-2: языковая модель, генератор текстов, состоящий только из блоков декодера. Все задачи решающиеся с помощью него сводятся к предсказанию следующего слова, для чего в модели используется маскированный self-attention (чтобы не знать правого контекста). Также используется BPE.

GPT-2 является самой большой моделью, и нет необходимости дообучать её для более специфичных задач, в отличие от BERT.

# Exam

Develop a model for predicting review rating.  
**Multiclass classification into 5 classes**  
Score: **F1 with macro averaging**  
You are forbidden to use test dataset for any kind of training.  
Remember proper training pipeline.  
If you are not using default params in the models, you have to use some validation scheme to justify them. 

Use `random_state` or `seed` params - your experiment must be reprodusible.


### 1 baseline = 0.51
### 2 baseline = 0.53


In [0]:
import numpy as np
import pandas as pd
import nltk

import torch as tt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchtext import data

from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [0]:
import pandas as pd

In [25]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.shape, df_test.shape

((48192, 3), (5355, 3))

Немного уменьшим количество данных, убрав тексты длинной больше 500 из трейн и тест сетов

In [0]:
def update(df):
  all_len = []
  for row in df.review:
    text_len = len(row)
    all_len.append(text_len)
  
  df['text_len'] = all_len

In [0]:
update(df_train)
df_train = df_train[df_train.text_len < 500].reset_index(drop=True)
df_train.drop('text_len', axis='columns', inplace=True)
df_train['target'] = df_train['target'].apply(float)

In [31]:
df_train.head()

,review,title,target
0,"The staff was very friendly, the breakfast ver...",Walker Gem,5.0
1,Excellent service - very approachable and prof...,Excellent Service,4.0
2,Really a top notch place to spend a day at the...,"Good location, warm and friendly staff",5.0
3,"a little noisy, there was a false fire alarm a...","nice hotel,",4.0
4,Place had too many animals and I'm allergic to...,Experience,3.0


In [55]:
update(df_test)
df_test = df_test[df_test.text_len < 500].reset_index(drop=True)
df_test.drop('text_len', axis='columns', inplace=True)
df_test['target'] = df_test['target'].apply(float)
df_test.head()

,review,title,target
0,Perfect place for a quick get away. We had a q...,Love this place!,5.0
1,"The room was not the best however, it was good...",Good For One Night Stay...,2.0
2,Sous le motif d'une priode hivernale (inaccept...,Moyen,3.0
3,This was our first time at the hotel. We will ...,Great for the whole family! :-),5.0
4,Stayed here last night for an overnight stay. ...,Overnight stay,4.0


Для эмбеддингов загрузим модель fasttext

In [40]:
pip install sister

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 21.5MB 221kB/s 
  Created wheel for sister: filename=sister-0.1.5-cp36-none-any.whl size=4232 sha256=39f40dd194c065c1fa0976f9308e7f5327622d6556e1449ca754ed9899cd1bb4
  Stored in directory: /root/.cache/pip/wheels/7d/ed/48/76cdf61511496a1cd2a2c56dc9b4e9667401dfb6084f153cc3
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2382664 sha256=c4490f8b155b83e39b12067cc3c9bceb595b776e1e11b543324a1b08e15a3ad5
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built sister fasttext


In [41]:
import sister
embedder = sister.MeanEmbedding(lang="en")

Loading model...


In [45]:
emb = embedder(df_train.review[0])
emb.size, emb[:5]

(300, array([ 0.17256983,  0.07729055,  0.15882157, -0.2502666 , -0.04762736],
       dtype=float32))

Возьмем из трейн сета 15000 текстов, из тест сета - все

In [0]:
df_train_2 = df_train[:15000]

In [57]:
df_train_2['embs'] = df_train_2['review'].apply(embedder)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
df_train_2.head()

,review,title,target,embs
0,"The staff was very friendly, the breakfast ver...",Walker Gem,5.0,"[0.17256983, 0.07729055, 0.15882157, -0.250266..."
1,Excellent service - very approachable and prof...,Excellent Service,4.0,"[0.17130384, 0.012917038, 0.16785558, -0.23492..."
2,Really a top notch place to spend a day at the...,"Good location, warm and friendly staff",5.0,"[0.14789103, 0.015175829, 0.13551584, -0.22039..."
3,"a little noisy, there was a false fire alarm a...","nice hotel,",4.0,"[0.123614825, -0.023343822, 0.05292964, -0.277..."
4,Place had too many animals and I'm allergic to...,Experience,3.0,"[0.1796624, 0.007334275, 0.10577045, -0.186901..."


In [59]:
df_test['embs'] = df_test['review'].apply(embedder)
df_test.head()

,review,title,target,embs
0,Perfect place for a quick get away. We had a q...,Love this place!,5.0,"[0.13594373, 0.014327291, 0.130299, -0.1782347..."
1,"The room was not the best however, it was good...",Good For One Night Stay...,2.0,"[0.094223924, 0.04474708, 0.12727132, -0.14162..."
2,Sous le motif d'une priode hivernale (inaccept...,Moyen,3.0,"[0.1909953, 0.025988193, -0.07697767, -0.09105..."
3,This was our first time at the hotel. We will ...,Great for the whole family! :-),5.0,"[0.13718049, 0.034042034, 0.14690219, -0.12456..."
4,Stayed here last night for an overnight stay. ...,Overnight stay,4.0,"[0.11817722, 0.035728753, 0.09630034, -0.21328..."


В качестве модели я выбрала модель с triplet loss из 8-ого дз, поэтому добавим еще положительные и отрицательные сэмплы

In [67]:
def pos_neg(row, df):
  pos = np.random.choice(df[df['target'] == row['target']]['embs'])
  neg = np.random.choice(df[df['target'] != row['target']]['embs'])

  return pos, neg

pos_vars = []
neg_vars = []

for i, row in df_train_2.iterrows():
  pos, neg = pos_neg(row, df_train_2)
  pos_vars.append(pos)
  neg_vars.append(neg)

df_train_2["pos"] = pos_vars
df_train_2["neg"] = neg_vars
df_train_2.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,review,title,target,embs,pos,neg
0,"The staff was very friendly, the breakfast ver...",Walker Gem,5.0,"[0.17256983, 0.07729055, 0.15882157, -0.250266...","[0.15477246, -0.035363674, 0.10628897, -0.2135...","[0.082609855, 0.014488304, 0.10910928, -0.1377..."
1,Excellent service - very approachable and prof...,Excellent Service,4.0,"[0.17130384, 0.012917038, 0.16785558, -0.23492...","[0.17312926, 0.038979106, 0.18059544, -0.21665...","[0.12519081, -0.010336755, 0.16430363, -0.2085..."
2,Really a top notch place to spend a day at the...,"Good location, warm and friendly staff",5.0,"[0.14789103, 0.015175829, 0.13551584, -0.22039...","[0.08580345, 0.07067273, 0.12666474, -0.125248...","[0.07966322, -0.05119884, 0.01786722, -0.24619..."
3,"a little noisy, there was a false fire alarm a...","nice hotel,",4.0,"[0.123614825, -0.023343822, 0.05292964, -0.277...","[0.17101215, 0.119695865, 0.007591687, -0.2145...","[0.16948985, 0.013746265, 0.14364307, -0.21542..."
4,Place had too many animals and I'm allergic to...,Experience,3.0,"[0.1796624, 0.007334275, 0.10577045, -0.186901...","[0.122860126, -0.0015181602, 0.107187636, -0.1...","[0.11987962, 0.014511491, 0.12315744, -0.20416..."


In [68]:
pos_vars = []
neg_vars = []

for i, row in df_test.iterrows():
  pos, neg = pos_neg(row, df_test)
  pos_vars.append(pos)
  neg_vars.append(neg)

df_test["pos"] = pos_vars
df_test["neg"] = neg_vars
df_test.head()

,review,title,target,embs,pos,neg
0,Perfect place for a quick get away. We had a q...,Love this place!,5.0,"[0.13594373, 0.014327291, 0.130299, -0.1782347...","[0.16547193, -0.03378663, 0.17914052, -0.13160...","[0.11891581, -0.0010144742, 0.10883977, -0.218..."
1,"The room was not the best however, it was good...",Good For One Night Stay...,2.0,"[0.094223924, 0.04474708, 0.12727132, -0.14162...","[0.14781778, 0.02219971, 0.12709615, -0.183789...","[0.11801362, 0.012230294, 0.1247987, -0.181724..."
2,Sous le motif d'une priode hivernale (inaccept...,Moyen,3.0,"[0.1909953, 0.025988193, -0.07697767, -0.09105...","[0.12776928, 0.015419931, 0.124923795, -0.1835...","[0.105306216, 0.035546087, 0.10962759, -0.1885..."
3,This was our first time at the hotel. We will ...,Great for the whole family! :-),5.0,"[0.13718049, 0.034042034, 0.14690219, -0.12456...","[0.12765536, 0.014298425, 0.08361171, -0.11456...","[0.12356367, -0.011370081, 0.19006196, -0.2429..."
4,Stayed here last night for an overnight stay. ...,Overnight stay,4.0,"[0.11817722, 0.035728753, 0.09630034, -0.21328...","[0.15000072, 0.13471246, 0.034320615, -0.10489...","[0.15127003, -0.005135447, 0.10767978, -0.1984..."


Поделим данные на три выборки

In [0]:
df_tr, df_val = train_test_split(df_train_2, test_size=0.1, random_state=42)

In [70]:
print('Train shape = ', df_tr.shape)
print('Validation shape = ', df_val.shape)
print('Test shape = ', df_test.shape)

Train shape =  (13500, 6)
Validation shape =  (1500, 6)
Test shape =  (4492, 6)


In [0]:
def triples(df):
  anch = list(df.embs)
  pos = list(df.pos)
  neg = list(df.neg)
  y = list(df.target)

  return tt.FloatTensor(anch), tt.FloatTensor(pos), tt.FloatTensor(neg), tt.FloatTensor(y)

In [0]:
anch_train, pos_train, neg_train, y_train = triples(df_tr)
anch_val, pos_val, neg_val, y_val = triples(df_val)
anch_test, pos_test, neg_test, y_test = triples(df_test)

In [0]:
batch_size = 32

train_loader = DataLoader(TensorDataset(anch_train, pos_train, neg_train,  y_train), batch_size=batch_size)
valid_loader = DataLoader(TensorDataset(anch_val, pos_val, neg_val,  y_val), batch_size=batch_size)
test_loader = DataLoader(TensorDataset(anch_test, pos_test, neg_test,  y_test), batch_size=batch_size)

In [0]:
def _train_epoch(model, iterator, optimizer, curr_epoch):

    model.train()

    running_loss = 0

    n_batches = len(iterator)
    iterator = tqdm_notebook(iterator, total=n_batches, desc='epoch %d' % (curr_epoch), leave=True)

    for i, batch in enumerate(iterator):
        optimizer.zero_grad()

        loss = model(batch)
        loss.backward()
        optimizer.step()

        curr_loss = loss.data.cpu().detach().item()
        
        loss_smoothing = i / (i+1)
        running_loss = loss_smoothing * running_loss + (1 - loss_smoothing) * curr_loss

        iterator.set_postfix(loss='%.5f' % running_loss)

    return running_loss

In [0]:
def _test_epoch(model, iterator):
    model.eval()
    epoch_loss = 0

    n_batches = len(iterator)
    with tt.no_grad():
        for batch in iterator:
            loss = model(batch)
            epoch_loss += loss.data.item()

    return epoch_loss / n_batches

In [0]:
def nn_train(model, train_iterator, valid_iterator, optimizer, n_epochs=10,
          scheduler=None, early_stopping=0):

    prev_loss = 100500
    es_epochs = 0
    best_epoch = None
    history = pd.DataFrame()

    for epoch in range(n_epochs):
        train_loss = _train_epoch(model, train_iterator, optimizer, epoch)
        valid_loss = _test_epoch(model, valid_iterator)

        valid_loss = valid_loss
        print('validation loss %.5f' % valid_loss)

        record = {'epoch': epoch, 'train_loss': train_loss, 'valid_loss': valid_loss}
        history = history.append(record, ignore_index=True)

        if early_stopping > 0:
            if valid_loss > prev_loss:
                es_epochs += 1
            else:
                es_epochs = 0

            if es_epochs >= early_stopping:
                best_epoch = history[history.valid_loss == history.valid_loss.min()].iloc[0]
                print('Early stopping! best epoch: %d val %.5f' % (best_epoch['epoch'], best_epoch['valid_loss']))
                break

            prev_loss = min(prev_loss, valid_loss)

Модель и функция потерь

In [0]:
def triplet_loss(anchor_embed, pos_embed, neg_embed, margin = 0.1):
  res = F.cosine_similarity(anchor_embed, neg_embed) - F.cosine_similarity(anchor_embed, pos_embed) + margin
  return tt.mean(res)

In [0]:
class Tripletnet(nn.Module):
    def __init__(self):
        super(Tripletnet, self).__init__()
        self.fc = nn.Linear(300, 128)
        
    def branch(self, x):
        x = self.fc(x)
        return x

    def forward(self, batch):

        anchor, pos, neg = batch[0], batch[1], batch[2]

        anchor = self.branch(anchor)
        pos = self.branch(pos)
        neg = self.branch(neg)
        
        return triplet_loss(anchor, pos, neg)

In [126]:
model_2 = Tripletnet()
optimizer = optim.Adam(model_2.parameters())
nn_train(model_2, train_loader, valid_loader, optimizer, n_epochs=5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



validation loss -0.15714



validation loss -0.16133



validation loss -0.24624



validation loss -0.24754



validation loss -0.25807


Попробуем оценить точность предсказаний. Воспользуемся методом ближайших соседей.

In [0]:
preds_2 = []
for i in X_test:
  preds_2.append(list(model_2.branch(i).detach().numpy()))

In [0]:
from sklearn.neighbors import NearestNeighbors
nns = NearestNeighbors(n_neighbors=3).fit(preds_2)
dist, inds = nns.kneighbors(preds_2)

In [0]:
preds_final_2 = []
for idx, vec in enumerate(preds_2):
    dists_all = [25]*25
    dist_vec = dist[idx]
    ind_vec = inds[idx]

    for i, el in enumerate(ind_vec):
        y_true = int(y_test[el].item())
        t = dist_vec[i]+dists_all[y_true]
        if dists_all[y_true] == 25:
          dists_all[y_true] = dist_vec[i]
        else:
          dists_all[y_true] = t
          
    y_pred = np.argmin(np.array(dists_all))
    preds_final_2.append(y_pred)

In [136]:
from sklearn.metrics import f1_score
f1_score(y_test, preds_final_2, average = 'macro')

0.8192510654697699